In [ ]:
from mesa import Agent, Model

from mesa.space import SingleGrid

from mesa.time import RandomActivation

from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd

import seaborn as sns

## AGENTES
### Explorer
- Buscar comida y almacén
- Convertir el valor de comida de 0 a 1
- Guardar y notificar la posición y cantidad de comida encontrada
### Collector
- Ir por la comida en un ruta definida
- Recoger la comida
- Dejar la comida en el almacén

In [ ]:
# Agent Class
class ExplorerAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)
        self.foundFood = False
        self.hasStorage = False

    def step(self):

        unfound_food = 47 - self.model.sum_food()

        if unfound_food > 0:
            self.lookforfood()
        else:
            pass

        if self.hasStorage:
            self.lookforfood()

    def move(self):
        neighbors = self.model.grid.get_neighborhood(self.pos, 
                                                     moore = True, 
                                                     include_center = False
        )

        is_possible = [step for step in neighbors if self.model.grid.is_cell_empty(step)]

        if is_possible:
            new_position = self.random.choice(is_possible)
            self.model.grid.move_agent(self, new_position)

    def lookforstorage(self):
        if not self.hasStorage:
            neighbors = self.model.grid.get_neighborhood(self.pos, 
                                                         moore = True, 
                                                         include_center = False
            )

            for neighbor in neighbors:
                if isinstance(self.model.grid.get_cell_list_contents(neighbor)[0]):
                    self.hasStorage = True
                    break

        

In [ ]:
class CollectorAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)
        self.hasFood = False

    def pickup(self):
        pass

    def drop(self):
        neighbors = self.model.grid.get_neighborhood(self.pos, 
                                                     moore = True, 
                                                     include_center = False
        )

        for neighbor in neighbors:
            x, y = neighbor_pos
            if self.model.grid[x][y].type == "storage":
                self.model.grid[x][y].food -=1
                self.hasFood = False
        self.move()

    def step(self):
        if self.hasFood:
            self.drop()
        else:
            self.pickup()

    def move(self):
        neighbors = self.model.grid.get_neighborhood(self.pos, 
                                                     moore = True, 
                                                     include_center = False
        )

        is_possible = [step for step in neighbors if self.model.grid.is_cell_empty(step)]

        if is_possible:
            new_position = self.random.choice(is_possible)
            self.model.grid.move_agent(self, new_position)

In [ ]:
# Model Class
class FoodModel(Model):
    def __init__(self, width, height, num_explorers, num_collectors):
        self.num_explorers = num_explorers
        self.num_collectors = num_collectors
        self.count_food = 47
        self.positions_food = []
        self.position_storage = []

        self.schedule = RandomActivation(self)
        self.grid = SingleGrid(width, height, torus = False)
        self.floor = np.zeros((width, height))

        self.datacollector = DataCollector(
            agent_reporters = {"hasFood": "hasFood", "hasStorage": "hasStorage"},
            model_reporters = {"Floor": self.get_floor, "Position": self.get_position}
        )

    def sum_found(self):
        found_food = 0
        for x in range(self.floor.shape[0]):
            for y in range(self.floor.shape[1]):
                if self.floor[x,y] <= 5:
                    found_food += 1
        return found_food
    
    def sum_collected(self):
        collected_food = 0
        for x in range(self.floor.shape[0]):
            for y in range(self.floor.shape[1]):
                if self.floor[x,y] > 0:
                    found_food += 1